# Verarbeitung, Analyse und Darstellung von Daten 

Python Bibliotheken (alle eignen sich hervorragend für die Verarbeitung sehr großer Datensätze):

Schnelle Datenanalyse und -manipulation - Pandas:
https://pandas.pydata.org/docs/

Bibliothek zur Datenvisualisierung insbesondere für statistische Grafiken - Seaborn:
https://seaborn.pydata.org/

Leistungsstarke Bibliothek für wissenschaftliches Rechnen - NumPy:
https://numpy.org/

Leistungsstarke Bibliothek mit grundlegenden Algorithmen für wissenschaftliches Rechnen:
https://scipy.org/

Datensatz der ZAMG - Zentralanstalt für Meteorologie und Geodynamik:
https://data.hub.zamg.ac.at/

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_theme(style="whitegrid")
import scipy.stats as sp
import matplotlib.pyplot as plt

Daten lesen

In [ ]:
pfad = "ZAMG_GRAZ_Datensatz_20000101_20221207.csv"
df=pd.read_csv(pfad, parse_dates=['time'])
df=df.set_index('time')
df.head()

Daten bereinigen

In [ ]:
df.dropna(axis=1, how='any')

In [ ]:
df

Was bedeuten die Kürzel? - Metadaten

In [ ]:
pfad_metadaten = "ZAMG_Parameter-Metadaten.csv"
df_meta=pd.read_csv(pfad_metadaten, quotechar = '*')
df_meta

In [ ]:
pd.set_option('display.max_colwidth', None)                        #Zeigt gesamte Spalte

In [ ]:
df_meta[df_meta["Identifier"]=="D6X"]

In [ ]:
df.columns

In [ ]:
df_meta[df_meta["Identifier"].isin(df.columns)]

Datenanalyse mit Pandas - deskriptive Statistik

In [ ]:
df.describe()

Wann gab es Niederschlag über 30mm pro Stunde?

In [ ]:
len(df[df['RSX']==0])

In [ ]:
len(df[df['RSX']<0.0000001])/165496

Visualisieren

In [ ]:
df[['TTX', 'TT3']]['2020-2':'2022-3'].plot()

In [ ]:
histogramm=sns.histplot([df['TTX']],binwidth=1, stat="count")
histogramm.set_xlabel("Temperatur")
histogramm.set_ylabel("Anzahl der Werte in der jeweiligen Klasse")

In [ ]:
histogramm=sns.histplot([df['TTX']],binwidth=1, stat="density")
histogramm.set_xlabel("Temperatur")
histogramm.set_ylabel("Relative Häufigkeitsdichte")

In [ ]:
histogramm=sns.histplot([df['TTX']],binwidth=1, stat="percent")
histogramm.set_xlabel("Temperatur")
histogramm.set_ylabel("Werte in der jeweiligen Klasse in Prozent")

Datenlücke

In [ ]:
df['TTX'].plot()

Zeitreihen Resampling - Datengranularität

In [ ]:
df_TTX = df['TTX'].resample('ME').mean()

In [ ]:
df_TTX['2020-2':'2022-3'].plot(marker='o')

In [ ]:
df_RSX = df['RSX'].resample('ME').sum()
df_RSX['2020-2':'2022-3'].plot(marker='o')

Daten speichern

In [ ]:
df_dymola = df.loc["2020-12"]

In [ ]:
df_dymola.LT2.plot()

In [ ]:
df_dymola["LT2"].fillna(0).plot()

In [ ]:
df_dymola["LT2"].bfill().plot()

In [ ]:
df_dymola["LT2"].ffill().plot()

In [ ]:
df_dymola["LT2"].interpolate(method="polynomial", order=2).plot()

In [ ]:
df_dymola = df_dymola.interpolate(method="polynomial", order=2)

In [ ]:
df.to_csv("ZAMG_GRAZ_Datensatz_20000101_20221207.txt")

Datenformat UNIX Zeitstempel: Anzahl der seit dem 1. Januar 1970, 00:00:00 UTC, verstrichenen Sekunden 

In [ ]:
df_dymola = df_dymola.assign(unix_timestamp = df_dymola.index)
df_dymola.unix_timestamp = df_dymola.unix_timestamp.apply(lambda x: x.timestamp())
df_dymola = df_dymola.set_index("unix_timestamp")

In [ ]:
df_dymola.to_csv("ZAMG_GRAZ_Datensatz_20000101_20221207.txt", sep=" ")

In [ ]:
f = open("ZAMG_GRAZ_Datensatz_20000101_20221207.txt","r+")
lines = f.readlines() # lese File
f.seek(0) # zurück zum Anfang
f.write("#1\n") # Schreibe neue Zeile
f.write("double ZAMGdata"+str(df_dymola.shape)+" #") # Schreibe neue Zeile
for line in lines: # Schreibe ursprünglichen Inhalt
    f.write(line)
f.close()

Datenanalyse - Linearer Fit

In [ ]:
y=np.array(df['TTX'].dropna().values, dtype=float)
x=np.array(pd.to_datetime(df['TTX'].dropna()).index.values, dtype=float)
slope, intercept, r_value, p_value, std_err =sp.linregress(x,y)
xf = np.linspace(min(x),max(x),100)
xf1 = xf.copy()
xf1 = pd.to_datetime(xf1)
yf = (slope*xf)+intercept

In [ ]:
f, ax = plt.subplots(1, 1)
df['TTX'].dropna().plot(ax=ax,marker='.', ls='')
ax.plot(xf1, yf,label='Linear fit', lw=3)
plt.ylabel('Temperatur')
ax.legend();

Wie viel ist die mittlere Lufttemperatur in 2m Höhe vor der Uni Graz seit 2000 gestiegen?

In [ ]:
yf[-1]-yf[0]

Regressionsanalyse und Ausreißer

In [ ]:
anscombe = sns.load_dataset("anscombe")

In [ ]:
sns.lmplot(x="x", y="y", data=anscombe.query("dataset == 'III'"), ci=None)

In [ ]:
sns.lmplot(x="x", y="y", data=anscombe.query("dataset == 'III'"), ci=None, robust=True)

#### Endenergieverbrauch in Österreich in TJ 1970-2021

Quelle: Statistik Austria (https://www.statistik.at/)


In [ ]:
energie = pd.read_excel("Energieendverbrauch.xlsx",skiprows=9,skipfooter=8,index_col=1)
energie.drop('Zeit', axis=1, inplace=True)
energie.drop('Bilanzaggregat', axis=0, inplace=True)
energie = energie.loc[energie.index.notnull()]
energie.drop('Energetischer Endverbrauch (EE)', axis=0, inplace=True)
energie = energie.transpose().sort_index().astype(float)
energie.index.name = 'Zeit [Jahre]'
energie.index = energie.index.astype(float)

In [ ]:
energie.head()

In [ ]:
energie.describe().transpose()

In [ ]:
energie.plot(figsize=(10,6),ylabel='Endenergieverbrauch [TJ]', title='Endenergieverbrauch Österreich 1970-2021', xlabel='Zeit [Jahre]').legend(loc='center left',bbox_to_anchor=(1.0, 0.5));

Boxplot

In [ ]:
boxplot = sns.boxplot(data=energie, orient='h')
boxplot.set_xlabel("Endenergieverbrauch [TJ]")